# Stepwise Analysis

Take genes from WTvex9 and WTvex20, then look at the differences between them

In [4]:
library("tidyverse")
library("org.Hs.eg.db")
library("ggrepel")
library("DESeq2")
setwd("/data/park_lab/adam/Total_RNA")
load("Total_RNA_env.RData")
ls()

In [ ]:
# save.image(file = "stepwise.RData")

In [ ]:
load("stepwise.RData")
ls()

## Aggregate

In [ ]:
WTvex9_genes_log2fc_1.5
WTvex20_genes_log2fc_1.5

In [ ]:
WTvex9_selected <- WTvex9_genes_log2fc_1.5 %>% 
                    dplyr::select("X", "symbol", "entrez")
WTvex20_selected <- WTvex20_genes_log2fc_1.5 %>% 
                    dplyr::select("X", "symbol", "entrez")

In [ ]:
head(WTvex9_selected)
head(WTvex20_selected)

interx <- WTvex9_selected %>% 
        filter(X %in% WTvex20_selected$X)
interx
# 28 genes are shared

In [ ]:
WTvex9_unique <- subset(WTvex9_selected, !(X %in% interx$X))
WTvex20_unique <- subset(WTvex20_selected, !(X %in% interx$X))

WTvex9_unique
WTvex20_unique

In [ ]:
total_counts

filter counts by the unique genes

In [ ]:
# unique_counts_9 <- total_counts %>% 
#                 filter(rownames(total_counts) %in% WTvex9_unique$X)
# unique_counts_9
# unique_counts_20 <- total_counts %>% 
#                 filter(rownames(total_counts) %in% WTvex20_unique$X)
# unique_counts_20
# aggregate_unique <- rbind(unique_counts_9, unique_counts_20)
# aggregate_unique
colnames(aggregate_unique)

In [ ]:
#Set treatment variables for DESeq
genotype <- c(rep("E545K",3), 
             rep("WT",6), 
             rep("E545K",3),
             rep("H1047R",3),
             rep("WT",3),
             rep("E545K",3),
             rep("H1047R",6))
genotype <- as.factor(genotype)
coldata <- data.frame(genotype)
rownames(coldata) <- colnames(aggregate_unique)
all(rownames(coldata) == colnames(aggregate_unique))

In [ ]:
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = aggregate_unique,
                              colData = coldata,
                              design = ~ genotype)

In [ ]:
dds

In [ ]:
dds <- DESeq(dds)
resultsNames(dds)

In [ ]:
######PCA for sample similarity
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "genotype")

In [ ]:
######## filter by significance
######## ## l2fc +/- 1.5 and padj <0.05
######## ###129 genes
split_res <- results(dds, contrast = c("genotype", "E545K", "H1047R"))
split_res <- as.data.frame(split_res)
split_res_df_filt <- split_res %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
split_res_df_filt

In [ ]:
library(tidyverse)
library(janitor)
library(scales)
library(DESeq2)
library(dplyr)
library(biomaRt)
library(ggrepel)
# library(clusterProfiler)
library(org.Hs.eg.db)

In [ ]:
#add gene symbols
split_res_df_filt$geneID <- rownames(split_res_df_filt) %>% 
  tools::file_path_sans_ext()
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
genes <- split_res_df_filt$geneID
G_list <- getBM(filters= "ensembl_gene_id",
                attributes= c("ensembl_gene_id","hgnc_symbol"),
                values=genes,mart= mart,
               useCache = FALSE)
split_res_df_filt <- merge(split_res_df_filt,
                               G_list,
                               by.x="geneID",
                               by.y="ensembl_gene_id")
write_tsv(split_res_df_filt, file = "Stepwise_aggregate_diff.txt")
head(split_res_df_filt)

In [ ]:
#volcano plot
split_res_df_filt$diffexpressed <- "NO"
split_res_df_filt$diffexpressed[split_res_df_filt$log2FoldChange > 1.5 & split_res_df_filt$padj < 0.05] <- "E545K"
split_res_df_filt$diffexpressed[split_res_df_filt$log2FoldChange < -1.5 & split_res_df_filt$padj < 0.05] <- "H1047R"
#volcano plot
volcano <- split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue"))
volcano

In [ ]:
#GOBP
agg_GOBP <- enrichGO

In [ ]:
browseVignettes("clusterProfiler")


## Individual Lines

In [ ]:
#Cancer
cancer_WTvex9_resSig_padj
cancer_WTvex20_resSig_padj

#H-Tert
HTert_WTvex9_resSig_padj
HTert_WTvex20_resSig_padj

#MCF10A
MCF10A_WTvex9_resSig_padj
MCF10A_WTvex20_resSig_padj

In [ ]:
#cancer
cancer_WTvex9_selected <- cancer_WTvex9_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
cancer_WTvex20_selected <- cancer_WTvex20_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
head(cancer_WTvex9_selected)
head(cancer_WTvex20_selected)
#H-Tert
HTert_WTvex9_selected <- HTert_WTvex9_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
HTert_WTvex20_selected <- HTert_WTvex20_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
head(HTert_WTvex9_selected)
head(HTert_WTvex20_selected)
#MCF10A
MCF10A_WTvex9_selected <- MCF10A_WTvex9_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
MCF10A_WTvex20_selected <- MCF10A_WTvex20_resSig_padj %>% 
                    dplyr::select("rowname", "log2FoldChange")
head(MCF10A_WTvex9_selected)
head(MCF10A_WTvex20_selected)

In [ ]:
cancer_interx <- cancer_WTvex9_selected %>% 
        filter(rowname %in% cancer_WTvex20_selected$rowname)
cancer_interx
# 548 genes are shared
HTert_interx <- HTert_WTvex9_selected %>% 
        filter(rowname %in% HTert_WTvex20_selected$rowname)
HTert_interx
#530
MCF10A_interx <- MCF10A_WTvex9_selected %>% 
        filter(rowname %in% MCF10A_WTvex20_selected$rowname)
MCF10A_interx
#410

In [ ]:
cancer_WTvex9_unique <- subset(cancer_WTvex9_selected, !(rowname %in% cancer_interx$rowname))
cancer_WTvex20_unique <- subset(cancer_WTvex20_selected, !(rowname %in% cancer_interx$rowname))
cancer_WTvex9_unique
#819
cancer_WTvex20_unique
#5431
HTert_WTvex9_unique <- subset(HTert_WTvex9_selected, !(rowname %in% HTert_interx$rowname))
HTert_WTvex20_unique <- subset(HTert_WTvex20_selected, !(rowname %in% HTert_interx$rowname))
HTert_WTvex9_unique
#1851
HTert_WTvex20_unique
#1555
MCF10A_WTvex9_unique <- subset(MCF10A_WTvex9_selected, !(rowname %in% MCF10A_interx$rowname))
MCF10A_WTvex20_unique <- subset(MCF10A_WTvex20_selected, !(rowname %in% MCF10A_interx$rowname))
MCF10A_WTvex9_unique
#750
MCF10A_WTvex20_unique
#1684
write_tsv(cancer_WTvex9_unique, file = "cancer_WTvex9_unique.txt")
write_tsv(cancer_WTvex20_unique, file = "cancer_WTvex20_unique.txt")
write_tsv(HTert_WTvex9_unique, file = "HTert_WTvex9_unique.txt")
write_tsv(HTert_WTvex20_unique, file = "HTert_WTvex20_unique.txt")
write_tsv(MCF10A_WTvex9_unique, file = "MCF10A_WTvex9_unique.txt")
write_tsv(MCF10A_WTvex20_unique, file = "MCF10A_WTvex20_unique.txt")

In [ ]:
ens.str <-substr(cancer_WTvex9_unique$rowname, 1,15) 
cancer_WTvex9_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
cancer_WTvex9_unique
ens.str <-substr(cancer_WTvex20_unique$rowname, 1,15) 
cancer_WTvex20_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
cancer_WTvex20_unique
ens.str <-substr(HTert_WTvex9_unique$rowname, 1,15) 
HTert_WTvex9_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
HTert_WTvex9_unique
ens.str <-substr(HTert_WTvex20_unique$rowname, 1,15) 
HTert_WTvex20_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
HTert_WTvex20_unique
ens.str <-substr(MCF10A_WTvex9_unique$rowname, 1,15) 
MCF10A_WTvex9_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
MCF10A_WTvex9_unique
ens.str <-substr(MCF10A_WTvex20_unique$rowname, 1,15) 
MCF10A_WTvex20_unique$symbol <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
MCF10A_WTvex20_unique

In [ ]:
cancer_WTvex9_unique <- na.omit(cancer_WTvex9_unique)
cancer_WTvex20_unique <- na.omit(cancer_WTvex20_unique)
HTert_WTvex9_unique <- na.omit(HTert_WTvex9_unique)
HTert_WTvex20_unique <- na.omit(HTert_WTvex20_unique)
MCF10A_WTvex9_unique <- na.omit(MCF10A_WTvex9_unique)
MCF10A_WTvex20_unique <- na.omit(MCF10A_WTvex20_unique)
cancer_WTvex9_unique
cancer_WTvex20_unique
HTert_WTvex9_unique
HTert_WTvex20_unique
MCF10A_WTvex9_unique
MCF10A_WTvex20_unique



In [ ]:
colnames(total_counts)

In [ ]:
cancer_counts <- total_counts %>%
                dplyr::select("MCF7_corrected_rep1", "MCF7_corrected_rep2", "MCF7_corrected_rep3",
                             "MCF7_parental_rep1", "MCF7_parental_rep2", "MCF7_parental_rep3",
                             "T47D_rep1", "T47D_rep2", "T47D_rep3")
HTert_counts <- total_counts %>%
                dplyr::select("HTert_WT_rep1", "HTert_WT_rep2", "HTert_WT_rep3",
                             "HTert_ex9_rep1", "HTert_ex9_rep2", "HTert_ex9_rep3",
                             "HTert_ex20_rep1", "HTert_ex20_rep2", "HTert_ex20_rep3")
MCF10A_counts <- total_counts %>%
                dplyr::select("MCF10A_parental_rep1", "MCF10A_parental_rep2", "MCF10A_parental_rep3",
                             "MCF10A_E545K_rep1", "MCF10A_E545K_rep2", "MCF10A_E545K_rep3",
                             "MCF10A_H1047R_rep1", "MCF10A_H1047R_rep2", "MCF10A_H1047R_rep3")

In [ ]:
cancer_unique_counts_9 <- cancer_counts %>% 
                filter(rownames(cancer_counts) %in% cancer_WTvex9_unique$rowname)
cancer_unique_counts_9
cancer_unique_counts_20 <- cancer_counts %>% 
                filter(rownames(cancer_counts) %in% cancer_WTvex20_unique$rowname)
cancer_unique_counts_20
cancer_unique_counts <- rbind(cancer_unique_counts_9, cancer_unique_counts_20)
cancer_unique_counts

HTert_unique_counts_9 <- HTert_counts %>% 
                filter(rownames(cancer_counts) %in% HTert_WTvex9_unique$rowname)
HTert_unique_counts_9
HTert_unique_counts_20 <- HTert_counts %>% 
                filter(rownames(HTert_counts) %in% HTert_WTvex20_unique$rowname)
HTert_unique_counts_20
HTert_unique_counts <- rbind(HTert_unique_counts_9, HTert_unique_counts_20)
HTert_unique_counts

MCF10A_unique_counts_9 <- MCF10A_counts %>% 
                filter(rownames(MCF10A_counts) %in% MCF10A_WTvex9_unique$rowname)
MCF10A_unique_counts_9
MCF10A_unique_counts_20 <- MCF10A_counts %>% 
                filter(rownames(MCF10A_counts) %in% MCF10A_WTvex20_unique$rowname)
MCF10A_unique_counts_20
MCF10A_unique_counts <- rbind(MCF10A_unique_counts_9, MCF10A_unique_counts_20)
MCF10A_unique_counts

In [ ]:
genotype <- c(rep("WT",3), 
             rep("E545K",3),
             rep("H1047R",3))

In [ ]:
genotype <- as.factor(genotype)
cancer_coldata <- data.frame(genotype)
HTert_coldata <- data.frame(genotype)
MCF10A_coldata <- data.frame(genotype)

rownames(cancer_coldata) <- colnames(cancer_unique_counts)
all(rownames(cancer_coldata) == colnames(cancer_unique_counts))

rownames(HTert_coldata) <- colnames(HTert_unique_counts)
all(rownames(HTert_coldata) == colnames(HTert_unique_counts))

rownames(MCF10A_coldata) <- colnames(MCF10A_unique_counts)
all(rownames(MCF10A_coldata) == colnames(MCF10A_unique_counts))

In [ ]:
#DESeq Time
cancer_dds <- DESeqDataSetFromMatrix(countData = cancer_unique_counts,
                              colData = cancer_coldata,
                              design = ~ genotype)
HTert_dds <- DESeqDataSetFromMatrix(countData = HTert_unique_counts,
                              colData = HTert_coldata,
                              design = ~ genotype)
MCF10A_dds <- DESeqDataSetFromMatrix(countData = MCF10A_unique_counts,
                              colData = MCF10A_coldata,
                              design = ~ genotype)

In [ ]:
cancer_dds <- DESeq(cancer_dds)
resultsNames(cancer_dds)
HTert_dds <- DESeq(HTert_dds)
resultsNames(HTert_dds)
MCF10A_dds <- DESeq(MCF10A_dds)
resultsNames(MCF10A_dds)

In [ ]:
cancer_dds_vst <- varianceStabilizingTransformation(cancer_dds, blind = FALSE)
plotPCA(cancer_dds_vst, intgroup = "genotype")

HTert_dds_vst <- varianceStabilizingTransformation(HTert_dds, blind = FALSE)
plotPCA(HTert_dds_vst, intgroup = "genotype")

MCF10A_dds_vst <- varianceStabilizingTransformation(MCF10A_dds, blind = FALSE)
plotPCA(MCF10A_dds_vst, intgroup = "genotype")

In [ ]:
######## filter by significance
######## ## l2fc +/- 1.5 and padj <0.05
######## ###129 genes
cancer_split_res <- results(cancer_dds, contrast = c("genotype", "E545K", "H1047R"))
cancer_split_res <- as.data.frame(cancer_split_res)
cancer_split_res_df_filt <- cancer_split_res %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
cancer_split_res_df_filt

HTert_split_res <- results(HTert_dds, contrast = c("genotype", "E545K", "H1047R"))
HTert_split_res <- as.data.frame(HTert_split_res)
HTert_split_res_df_filt <- HTert_split_res %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
HTert_split_res_df_filt

MCF10A_split_res <- results(MCF10A_dds, contrast = c("genotype", "E545K", "H1047R"))
MCF10A_split_res <- as.data.frame(MCF10A_split_res)
MCF10A_split_res_df_filt <- MCF10A_split_res %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
MCF10A_split_res_df_filt

In [ ]:
#add gene symbols
cancer_split_res_df_filt$geneID <- rownames(cancer_split_res_df_filt) %>% 
  tools::file_path_sans_ext()
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
genes <- cancer_split_res_df_filt$geneID
G_list <- getBM(filters= "ensembl_gene_id",
                attributes= c("ensembl_gene_id","hgnc_symbol"),
                values=genes,mart= mart,
               useCache = FALSE)
cancer_split_res_df_filt <- merge(cancer_split_res_df_filt,
                               G_list,
                               by.x="geneID",
                               by.y="ensembl_gene_id")
write_tsv(cancer_split_res_df_filt, file = "cancer_Stepwise_aggregate_diff.txt")
head(cancer_split_res_df_filt)

HTert_split_res_df_filt$geneID <- rownames(HTert_split_res_df_filt) %>% 
  tools::file_path_sans_ext()
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
genes <- HTert_split_res_df_filt$geneID
G_list <- getBM(filters= "ensembl_gene_id",
                attributes= c("ensembl_gene_id","hgnc_symbol"),
                values=genes,mart= mart,
               useCache = FALSE)
HTert_split_res_df_filt <- merge(HTert_split_res_df_filt,
                               G_list,
                               by.x="geneID",
                               by.y="ensembl_gene_id")
write_tsv(HTert_split_res_df_filt, file = "HTert_Stepwise_aggregate_diff.txt")
head(HTert_split_res_df_filt)

MCF10A_split_res_df_filt$geneID <- rownames(MCF10A_split_res_df_filt) %>% 
  tools::file_path_sans_ext()
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
genes <- MCF10A_split_res_df_filt$geneID
G_list <- getBM(filters= "ensembl_gene_id",
                attributes= c("ensembl_gene_id","hgnc_symbol"),
                values=genes,mart= mart,
               useCache = FALSE)
MCF10A_split_res_df_filt <- merge(MCF10A_split_res_df_filt,
                               G_list,
                               by.x="geneID",
                               by.y="ensembl_gene_id")
write_tsv(MCF10A_split_res_df_filt, file = "MCF10A_Stepwise_aggregate_diff.txt")
head(MCF10A_split_res_df_filt)

In [ ]:
#volcano plot
cancer_split_res_df_filt$diffexpressed <- "NO"
cancer_split_res_df_filt$diffexpressed[cancer_split_res_df_filt$log2FoldChange > 1.5 & cancer_split_res_df_filt$padj < 0.05] <- "E545K"
cancer_split_res_df_filt$diffexpressed[cancer_split_res_df_filt$log2FoldChange < -1.5 & cancer_split_res_df_filt$padj < 0.05] <- "H1047R"
#volcano plot
volcano <- cancer_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) +
    ggtitle(label = "cancer ex9 v ex20 from stepwise analysis")
volcano

In [ ]:
#volcano plot
HTert_split_res_df_filt$diffexpressed <- "NO"
HTert_split_res_df_filt$diffexpressed[HTert_split_res_df_filt$log2FoldChange > 1.5 & HTert_split_res_df_filt$padj < 0.05] <- "E545K"
HTert_split_res_df_filt$diffexpressed[HTert_split_res_df_filt$log2FoldChange < -1.5 & HTert_split_res_df_filt$padj < 0.05] <- "H1047R"
#volcano plot
volcano <- HTert_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "HTert ex9 v ex20 from stepwise analysis")
volcano

In [ ]:
#volcano plot
MCF10A_split_res_df_filt$diffexpressed <- "NO"
MCF10A_split_res_df_filt$diffexpressed[MCF10A_split_res_df_filt$log2FoldChange > 1.5 & MCF10A_split_res_df_filt$padj < 0.05] <- "E545K"
MCF10A_split_res_df_filt$diffexpressed[MCF10A_split_res_df_filt$log2FoldChange < -1.5 & MCF10A_split_res_df_filt$padj < 0.05] <- "H1047R"
#volcano plot
volcano <- MCF10A_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "MCF10A ex9 v ex20 from stepwise analysis")
volcano

### Shared

In [ ]:
shared_split_res_df_filt <- cancer_split_res_df_filt %>% 
                filter(geneID %in% HTert_split_res_df_filt$geneID)
shared_split_res_df_filt <- shared_split_res_df_filt %>%
                filter(geneID %in% MCF10A_split_res_df_filt$geneID)
shared_split_res_df_filt

In [ ]:
write_tsv(shared_split_res_df_filt, file = "shared_genes_step.txt")

In [ ]:
volcano <- shared_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "Shared ex9 v ex20 from stepwise analysis")
volcano

In [ ]:
pdf(file = "Volcanoes",
   width = 8,
   height = 6)
options(repr.plot.width=8, repr.plot.height=6)
volcano <- cancer_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) +
    ggtitle(label = "cancer ex9 v ex20 from stepwise analysis")
volcano
volcano <- HTert_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "HTert ex9 v ex20 from stepwise analysis")
volcano
volcano <- MCF10A_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "MCF10A ex9 v ex20 from stepwise analysis")
volcano
volcano <- shared_split_res_df_filt %>%
  ggplot(aes(x = log2FoldChange, y = -log10(padj), label = hgnc_symbol, col = diffexpressed)) +
  geom_point() +
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values = c("lightpink3", "cadetblue")) + 
    ggtitle(label = "Shared ex9 v ex20 from stepwise analysis")
volcano
dev.off()

In [ ]:
cd /data/park_lab/adam/Total_RNA

In [ ]:
cat Moar_analysis.R

#### Shared Directionality

In [ ]:
head(cancer_split_res_df_filt)
head(HTert_split_res_df_filt)
head(MCF10A_split_res_df_filt)

# shared_split_res_df_filt <- cancer_split_res_df_filt %>% 
#                 filter(rownames(cancer_split_res_df_filt) %in% rownames(HTert_split_res_df_filt))
# shared_split_res_df_filt <- shared_split_res_df_filt %>%
#                 filter(rownames(shared_split_res_df_filt) %in% rownames(MCF10A_split_res_df_filt))
# shared_split_res_df_filt

In [ ]:
cancer_split_res_df_filt_9 <- cancer_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange > 0)
cancer_split_res_df_filt_20 <- cancer_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange < 0)

HTert_split_res_df_filt_9 <- HTert_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange > 0)
HTert_split_res_df_filt_20 <- HTert_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange < 0)

MCF10A_split_res_df_filt_9 <- MCF10A_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange > 0)
MCF10A_split_res_df_filt_20 <- MCF10A_split_res_df_filt %>%
                                dplyr::filter(log2FoldChange < 0)

In [ ]:
cancer_split_res_df_filt_9
cancer_split_res_df_filt_20
HTert_split_res_df_filt_9
HTert_split_res_df_filt_20
MCF10A_split_res_df_filt_9
MCF10A_split_res_df_filt_20

In [ ]:
shared_split_res_df_filt_9 <- cancer_split_res_df_filt_9 %>% 
                filter(geneID %in% HTert_split_res_df_filt_9$geneID)
shared_split_res_df_filt_9 <- shared_split_res_df_filt_9 %>%
                filter(geneID %in% MCF10A_split_res_df_filt_9$geneID)
shared_split_res_df_filt_9

In [ ]:
shared_split_res_df_filt_20 <- cancer_split_res_df_filt_20 %>% 
                filter(geneID %in% HTert_split_res_df_filt_20$geneID)
shared_split_res_df_filt_20 <- shared_split_res_df_filt_9 %>%
                filter(geneID %in% MCF10A_split_res_df_filt_20$geneID)
shared_split_res_df_filt_20

## TGFB Specific Analysis of RNA-seq data

The goal is to filter the RNA-seq results for TGFB associated genes and perform dimensionality reduction to ideally stratify data by mutation status.

Set directory to folder with RNA-seq environment from laptop.

In [ ]:
library("tidyverse")
library("org.Hs.eg.db")
library("ggrepel")

In [ ]:
setwd("/data/park_lab/adam/Total_RNA")

Load that environment

In [ ]:
load("Total_RNA_env.RData")
ls()

In [ ]:
head(total_counts_t)
head(normalized_total_counts_t)

Read in the list of genes from the TGFB pathway (GO: 0071559)

In [ ]:
TGFB_genes <- read_tsv(file = "tgfb_gene_list_GO_0071559.txt")
head(TGFB_genes)
TGFB_genes <- rbind(names(TGFB_genes), TGFB_genes)
names(TGFB_genes) <- c("UniProtID", "Symbol")
head(TGFB_genes)

Add ENSEMBL IDs

In [ ]:
symbol.str <- substr(TGFB_genes$Symbol, 1,15)
symbol.str
TGFB_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
head(TGFB_genes)

### Raw Counts

Remove the Transcript ID from the Ensembl IDs in the counts data

In [ ]:
total_counts_t <- t(total_counts)
head(total_counts_t)
good_names <- colnames(total_counts_t)
good_names <- tools::file_path_sans_ext(c(good_names))
good_names
colnames(total_counts_t) <- good_names
head(total_counts_t)

Filter the counts data by the TGFB genes

In [ ]:
total_counts_t <- as.data.frame(total_counts_t)
TGFB_ensembl <- TGFB_genes$Ensembl %>%
                as.character()
TGFB_ensembl
TGFB_ensembl <- na.omit(TGFB_ensembl)
TGFB_ensembl <- as.character(TGFB_ensembl)
str(TGFB_ensembl)
TGFB_counts <- total_counts_t[, (colnames(total_counts_t) %in% c(TGFB_ensembl))]
head(TGFB_counts)

In [ ]:
ens.str <- substr(colnames(TGFB_counts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(TGFB_counts) <- genes_for_pca
colnames(TGFB_counts)

Principal Components

In [ ]:
library("factoextra")

In [ ]:
res.pca <- prcomp(TGFB_counts, scale = TRUE)

In [ ]:
fviz_eig(res.pca)

In [ ]:
fviz_pca_ind(res.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

In [ ]:
fviz_pca_var(res.pca,
             col.var = "contrib", # Color by contributions to the PC
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = FALSE     # Avoid text overlapping
             )

In [ ]:
fviz_pca_biplot(res.pca, repel = FALSE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

### Normalized Counts

In [ ]:
head(normalized_total_counts_t)
norm_names <- colnames(normalized_total_counts_t)
norm_names <- tools::file_path_sans_ext(c(norm_names))
norm_names
colnames(normalized_total_counts_t) <- norm_names
head(normalized_total_counts_t)

Filter for TGFB genes

In [ ]:
TGFB_ensembl <- TGFB_genes$Ensembl %>%
                as.character()
TGFB_ensembl
TGFB_ensembl <- na.omit(TGFB_ensembl)
TGFB_ensembl <- as.character(TGFB_ensembl)
str(TGFB_ensembl)
TGFB_normCounts <- normalized_total_counts_t[, (colnames(normalized_total_counts_t) %in% c(TGFB_ensembl))]
head(TGFB_normCounts)

In [ ]:
ens.str <- substr(colnames(TGFB_normCounts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(TGFB_normCounts) <- genes_for_pca
colnames(TGFB_normCounts)

In [ ]:
res.Normpca <- prcomp(TGFB_normCounts, scale = TRUE)

In [ ]:
fviz_eig(res.Normpca)

In [ ]:
fviz_pca_ind(res.Normpca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

In [ ]:
fviz_pca_var(res.Normpca,
             col.var = "contrib", # Color by contributions to the PC
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

In [ ]:
fviz_pca_biplot(res.Normpca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

## Hippo Pathway

Read in list of hippo pathway genes

In [ ]:
hippo_genes <- read_tsv(file = "hippo_gene_list_GO_0035329.txt")
head(hippo_genes)
hippo_genes <- rbind(names(hippo_genes), hippo_genes)
names(hippo_genes) <- c("UniProtID", "Name", "Symbol")
head(hippo_genes)

In [ ]:
symbol.str <- substr(hippo_genes$Symbol, 1,15)
symbol.str
hippo_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
hippo_genes

Filter the raw and normalized counts by the hippo pathway

In [ ]:
total_counts_t <- as.data.frame(total_counts_t)
hippo_ensembl <- hippo_genes$Ensembl %>%
                as.character()
hippo_ensembl
hippo_ensembl <- na.omit(hippo_ensembl)
hippo_ensembl <- as.character(hippo_ensembl)
str(hippo_ensembl)
hippo_counts <- total_counts_t[, (colnames(total_counts_t) %in% c(hippo_ensembl))]
head(hippo_counts)

In [ ]:
hippo_normCounts <- normalized_total_counts_t[, (colnames(normalized_total_counts_t) %in% 
                                                c(hippo_ensembl))]
head(hippo_normCounts)

Add back the gene names

In [ ]:
ens.str <- substr(colnames(hippo_counts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(hippo_counts) <- genes_for_pca
colnames(hippo_counts)

ens.str <- substr(colnames(hippo_normCounts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(hippo_normCounts) <- genes_for_pca
colnames(hippo_normCounts)

In [ ]:
hippo_res.pca <- prcomp(hippo_counts, scale = TRUE)
hippo_normres.pca <- prcomp(hippo_normCounts, scale = TRUE)

In [ ]:
fviz_pca_ind(hippo_res.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )
### Raw Counts

In [ ]:
fviz_pca_ind(hippo_normres.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )
### Normalized Counts

In [ ]:
fviz_pca_biplot(hippo_res.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )
fviz_pca_biplot(hippo_normres.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

## AP-1

In [ ]:
ap1_genes <- read_tsv(file = "ap1_gene_list_GO_0035976.txt")
head(ap1_genes)
hippo_genes <- rbind(names(ap1_genes), ap1_genes)
names(ap1_genes) <- c("UniProtID", "Name", "Symbol")
head(ap1_genes)

In [ ]:
symbol.str <- substr(ap1_genes$Symbol, 1,15)
symbol.str
ap1_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
ap1_genes

## PI3K Pathway

Probably the pathway I should've tried first

In [ ]:
pi3k_genes <- read_tsv(file = "pi3k_gene_list_GO_0014065.txt")
head(pi3k_genes)
hippo_genes <- rbind(names(pi3k_genes), pi3k_genes)
names(pi3k_genes) <- c("UniProtID", "Name", "Symbol")
head(pi3k_genes)

In [ ]:
symbol.str <- substr(pi3k_genes$Symbol, 1,15)
symbol.str
pi3k_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
pi3k_genes

In [ ]:
pi3k_ensembl <- pi3k_genes$Ensembl %>%
                as.character()
pi3k_ensembl
pi3k_ensembl <- na.omit(pi3k_ensembl)
pi3k_ensembl <- as.character(pi3k_ensembl)
pi3k_normCounts <- normalized_total_counts_t[, (colnames(normalized_total_counts_t) %in% 
                                                c(pi3k_ensembl))]
head(pi3k_normCounts)

In [ ]:
ens.str <- substr(colnames(pi3k_normCounts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(pi3k_normCounts) <- genes_for_pca
colnames(pi3k_normCounts)

In [ ]:
pi3k_normres.pca <- prcomp(pi3k_normCounts, scale = TRUE)

In [ ]:
library("factoextra")
fviz_pca_ind(pi3k_normres.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )
### Normalized Counts

In [ ]:
fviz_pca_biplot(pi3k_normres.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

## Growth Factor Response in general

In [ ]:
GF_genes <- read_tsv(file = "GF_gene_list_GO_0070848.txt")
head(GF_genes)
GF_genes <- rbind(names(GF_genes), GF_genes)
names(GF_genes) <- c("UniProtID", "Symbol", "Name")
head(GF_genes)

In [ ]:
symbol.str <- substr(GF_genes$Symbol, 1,15)
symbol.str
GF_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
GF_genes

In [ ]:
GF_ensembl <- GF_genes$Ensembl %>%
                as.character()
GF_ensembl
GF_ensembl <- na.omit(GF_ensembl)
GF_ensembl <- as.character(GF_ensembl)
GF_normCounts <- normalized_total_counts_t[, (colnames(normalized_total_counts_t) %in% 
                                                c(GF_ensembl))]
head(GF_normCounts)

In [ ]:
ens.str <- substr(colnames(GF_normCounts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(GF_normCounts) <- genes_for_pca
colnames(GF_normCounts)

In [ ]:
GF_normres.pca <- prcomp(GF_normCounts, scale = TRUE)

In [ ]:
library("factoextra")
fviz_pca_ind(GF_normres.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )
### Normalized Counts

In [ ]:
fviz_pca_biplot(GF_normres.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

## RTK Pathway

In [ ]:
RTK_genes <- read_tsv(file = "RTK_path_genes.txt")
head(RTK_genes)
RTK_genes <- rbind(names(RTK_genes), RTK_genes)
names(RTK_genes) <- c("UniProtID", "Name", "Symbol")
head(RTK_genes)

In [ ]:

symbol.str <- substr(RTK_genes$Symbol, 1,15)
symbol.str
RTK_genes$Ensembl <- mapIds(org.Hs.eg.db,
                            keys = symbol.str,
                            column = "ENSEMBL",
                            keytype = "SYMBOL",
                            multiVals= "first")
RTK_genes

In [ ]:
head(normalized_total_counts_t)
norm_names <- colnames(normalized_total_counts_t)
norm_names <- tools::file_path_sans_ext(c(norm_names))
norm_names
colnames(normalized_total_counts_t) <- norm_names
head(normalized_total_counts_t)

In [ ]:
RTK_ensembl <- RTK_genes$Ensembl %>%
                as.character()
RTK_ensembl
RTK_ensembl <- na.omit(RTK_ensembl)
RTK_ensembl <- as.character(RTK_ensembl)
RTK_ensembl
RTK_normCounts <- normalized_total_counts_t[, (colnames(normalized_total_counts_t) %in% 
                                                c(RTK_ensembl))]
head(RTK_normCounts)

In [ ]:
ens.str <- substr(colnames(RTK_normCounts), 1,15)
genes_for_pca <- mapIds(org.Hs.eg.db,
                            keys = ens.str,
                            column = "SYMBOL",
                            keytype = "ENSEMBL",
                            multiVals= "first")
str(genes_for_pca)
genes_for_pca <- as.character(genes_for_pca)
genes_for_pca
colnames(RTK_normCounts) <- genes_for_pca
colnames(RTK_normCounts)

In [ ]:
RTK_normres.pca <- prcomp(RTK_normCounts, scale = TRUE)

In [ ]:
library("factoextra")
fviz_pca_ind(RTK_normres.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )
### Normalized Counts

In [ ]:
fviz_pca_biplot(RTK_normres.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

# New NEW Approach: Remove Lineage-specific genes

New approach from Wendy. Let's try to identify lineage-specific expression markers and eliminate those from the DESeq2 analysis.

In [ ]:
total_counts

In [ ]:
names(total_counts)

In [ ]:
lineage <- c(rep("MCF10A",6), 
             rep("cancer",6), 
             rep("MCF10A",3),
             rep("HTert",9),
             rep("cancer",3))
lineage <- as.factor(lineage)
coldata <- data.frame(lineage)
rownames(coldata) <- names(total_counts)
all(rownames(coldata) == colnames(total_counts))

In [ ]:
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = total_counts,
                              colData = coldata,
                              design = ~ lineage)

In [ ]:
dds

In [ ]:
dds <- DESeq(dds)
resultsNames(dds)

In [ ]:
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "lineage")

In [ ]:
######## filter by significance
######## ## l2fc +/- 1.5 and padj <0.05
######## ###12895 genes (1.5fc)
#10618 (2fc)
#8936 (2.5fc)
canvHTert_split_res <- results(dds, contrast = c("lineage", "cancer", "HTert"))
canvHTert_split_res <- as.data.frame(canvHTert_split_res)
canvHTert_split_res_df_filt <- canvHTert_split_res %>%
  filter(log2FoldChange < -2.5 | log2FoldChange > 2.5) %>%
  filter(padj < 0.05)
canvHTert_split_res_df_filt

#11401 genes (1.5fc)
#8930 genes (2fc)
#7127 genes (2.5 fc)
canvMCF10A_split_res <- results(dds, contrast = c("lineage", "cancer", "MCF10A"))
canvMCF10A_split_res <- as.data.frame(canvMCF10A_split_res)
canvMCF10A_split_res_df_filt <- canvMCF10A_split_res %>%
  filter(log2FoldChange < -2.5 | log2FoldChange > 2.5) %>%
  filter(padj < 0.05)
canvMCF10A_split_res_df_filt

#9642 genes (1.5fc)
#7771 genes (2fc)
#6394 genes (2.5fc)
HTertvMCF10A_split_res <- results(dds, contrast = c("lineage", "HTert", "MCF10A"))
HTertvMCF10A_split_res <- as.data.frame(HTertvMCF10A_split_res)
HTertvMCF10A_split_res_df_filt <- HTertvMCF10A_split_res %>%
  filter(log2FoldChange < -2.5 | log2FoldChange > 2.5) %>%
  filter(padj < 0.05)
HTertvMCF10A_split_res_df_filt

In [ ]:
#At 2.5fc threshold
lineage_diff_genes <- c(c(rownames(canvHTert_split_res_df_filt), 
                          rownames(canvMCF10A_split_res_df_filt)), 
                          rownames(HTertvMCF10A_split_res_df_filt))
lineage_diff_genes
str(lineage_diff_genes)
lineage_diff_genes <- unique(lineage_diff_genes)
str(lineage_diff_genes)

In [ ]:
`%!in%` = Negate(`%in%`)
NonLineage_counts <- total_counts %>% 
                filter(rownames(total_counts) %!in% lineage_diff_genes)
NonLineage_counts

## Removed Non-lineage Counts

### By Genotype

In [ ]:
genotype <- c(rep("E545K",3), 
             rep("WT",6), 
             rep("E545K",3),
             rep("H1047R",3),
             rep("WT",3),
             rep("E545K",3),
             rep("H1047R",6))
genotype <- as.factor(genotype)
coldata <- data.frame(genotype)
rownames(coldata) <- colnames(NonLineage_counts)
all(rownames(coldata) == colnames(NonLineage_counts))

In [ ]:
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = NonLineage_counts,
                              colData = coldata,
                              design = ~ genotype)

In [ ]:
dds

In [ ]:
dds <- DESeq(dds)
resultsNames(dds)

In [ ]:
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "genotype")

### By lineage

In [ ]:
lineage <- c(rep("MCF10A",6), 
             rep("cancer",6), 
             rep("MCF10A",3),
             rep("HTert",9),
             rep("cancer",3))
lineage <- as.factor(lineage)
coldata <- data.frame(lineage)
rownames(coldata) <- names(NonLineage_counts)
all(rownames(coldata) == colnames(NonLineage_counts))
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = NonLineage_counts,
                              colData = coldata,
                              design = ~ lineage)
dds <- DESeq(dds)
resultsNames(dds)
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "lineage")

## Throw out the Cancer cells

Because Fuck 'em

In [ ]:
names(total_counts)

In [ ]:
nocan_names <- c("MCF10A_E545K_rep1", "MCF10A_E545K_rep2", "MCF10A_E545K_rep3",
                "MCF10A_parental_rep1", "MCF10A_parental_rep2", "MCF10A_parental_rep3",
                "MCF10A_H1047R_rep1", "MCF10A_H1047R_rep2", "MCF10A_H1047R_rep3", 
                "HTert_WT_rep1", "HTert_WT_rep2", "HTert_WT_rep3", 
                "HTert_ex9_rep1", "HTert_ex9_rep2", "HTert_ex9_rep3", 
                "HTert_ex20_rep1", "HTert_ex20_rep2", "HTert_ex20_rep3")
str(nocan_names)

In [ ]:
nocan_counts <- total_counts %>%
                dplyr::select(nocan_names)
nocan_counts

### Remove Lineage Specific Genes

In [ ]:
lineage <- c(rep("MCF10A",9),
             rep("HTert",9))
lineage <- as.factor(lineage)
coldata <- data.frame(lineage)
rownames(coldata) <- names(nocan_counts)
all(rownames(coldata) == colnames(nocan_counts))

In [ ]:
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = nocan_counts,
                              colData = coldata,
                              design = ~ lineage)
dds
dds <- DESeq(dds)
resultsNames(dds)

In [ ]:
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "lineage")

In [ ]:
#9580 genes at 1.5fc
#6284 genes at 2.5fc
nocan_HTertvMCF10A_split_res <- results(dds, contrast = c("lineage", "HTert", "MCF10A"))
nocan_HTertvMCF10A_split_res <- as.data.frame(nocan_HTertvMCF10A_split_res)
nocan_HTertvMCF10A_split_res_df_filt <- nocan_HTertvMCF10A_split_res %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
nocan_HTertvMCF10A_split_res_df_filt

In [ ]:
`%!in%` = Negate(`%in%`)
nocan_NonLineage_counts <- nocan_counts %>% 
                filter(rownames(nocan_counts) %!in% rownames(nocan_HTertvMCF10A_split_res_df_filt))
nocan_NonLineage_counts

In [ ]:
names(nocan_NonLineage_counts)

### No cancer cells, lineage specific genes removed

In [ ]:
genotype <- c(rep("E545K",3), 
             rep("WT",3),
             rep("H1047R",3),
             rep("WT",3),
             rep("E545K",3),
             rep("H1047R",3))
genotype <- as.factor(genotype)
coldata <- data.frame(genotype)
rownames(coldata) <- colnames(nocan_NonLineage_counts)
all(rownames(coldata) == colnames(nocan_NonLineage_counts))

In [ ]:
#DESeq Time
dds <- DESeqDataSetFromMatrix(countData = nocan_NonLineage_counts,
                              colData = coldata,
                              design = ~ genotype)
dds
dds <- DESeq(dds)
resultsNames(dds)
dds_vst <- varianceStabilizingTransformation(dds, blind = FALSE)
plotPCA(dds_vst, intgroup = "genotype")

# Shared Pathways

First we gotta do lineage independent comparisons and KEGG enrichment for each mutational comparison

In [ ]:
cat SharedPaths.R

In [ ]:
library("tidyverse")
library("org.Hs.eg.db")
library("ggrepel")
library("DESeq2")
setwd("/data/park_lab/adam/Total_RNA")

In [ ]:
load(file = "sharedPaths.RData")

In [ ]:
ls()

In [ ]:
sharedPaths_df

In [ ]:
WTvex20_sharedPaths_df

# Linear Model?

In [ ]:
library("tidyverse")
library("org.Hs.eg.db")
library("ggrepel")
library("DESeq2")
setwd("/data/park_lab/adam/Total_RNA")
load("Total_RNA_env.RData")
ls()

In [ ]:
normalized_total_counts_t

Add Categorical Columns for model to incorporate

In [ ]:
Lineage <- c(rep("MCF10A", 6),
             rep("cancer", 6),
             rep("MCF10A", 3),
             rep("HTert", 9),
             rep("cancer", 3))
Lineage

In [ ]:
Genotype <- c(rep("ex9", 3), rep("WT", 6), rep("ex9", 3), rep("ex20", 3), 
             rep("WT", 3), rep("ex9", 3), rep("ex20", 6))
Genotype

In [ ]:
norm_counts_model <- normalized_total_counts_t
norm_counts_model <- cbind(Lineage, Genotype, norm_counts_model)
norm_counts_model

GLM loop from Kevin Blighe, Adding covariate of lineage

In [ ]:
formula.start <- "Genotype ~ "
covars <- " + Lineage"
listModels <- list()

for (i in 3:ncol(norm_counts_model))
{
    formula <- paste(formula.start, colnames(norm_counts_model)[i], covars, sep="")
    model <- glm(as.formula(formula), data=norm_counts_model, family=binomial(link="logit"))
    listModels[[i]] <- model
    names(listModels[i]) <- colnames(norm_counts_model)[i]
    print(summary(model))
}

In [ ]:
listModels

In [ ]:
save.image(file = "GLM.RData")

# Modeling Tom's Way

Based on Tom's code:

Saved at:
/data/park_lab/adam/rclass.final.Rmd

In [1]:
library("tidyverse")
library("org.Hs.eg.db")
library("ggrepel")
library("DESeq2")
library("vsn")
library("RColorBrewer")
library("AnnotationDbi")
library("org.Hs.eg.db")
library("ReportingTools")
library("sva")
library("GSVA")
library("GSEABase")
library("Biobase")
library("genefilter")
library("gplots")
library("pheatmap")
library("limma")
library("stats")
library("grid")
library("pathfindR")
library("magick")
setwd("/data/park_lab/adam/Total_RNA")
load("model_env.RData")
ls()

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks jsonlite::flatten()
x dplyr::lag()     masks stats::lag()

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, i

[1] "AREG_exp"                                 
  [2] "AREG_exp_raw"                             
  [3] "CRISPR_guides"                            
  [4] "E545K_expressed_E545K_interacting_overlap"
  [5] "E545K_expressed_WT_interacting_overlap"   
  [6] "E545K_expressed_interacting_overlap"      
  [7] "ETV5_exp"                                 
  [8] "G_list"                                   
  [9] "Gene_targets"                             
 [10] "Genotype_vec"                             
 [11] "HTertDists"                               
 [12] "HTert_WT_counts"                          
 [13] "HTert_WTvex20_over"                       
 [14] "HTert_WTvex20_res"                        
 [15] "HTert_WTvex20_resSig_padj"                
 [16] "HTert_WTvex20_under"                      
 [17] "HTert_WTvex9_over"                        
 [18] "HTert_WTvex9_res"                         
 [19] "HTert_WTvex9_resSig_padj"                 
 [20] "HTert_WTvex9_under"                       
 [21] "HTert_coldata"                            
 [22] "HTert_counts"                             
 [23] "HTert_dds"                                
 [24] "HTert_dds_f"                              
 [25] "HTert_diff_genes"                         
 [26] "HTert_ex20_counts"                        
 [27] "HTert_ex9_counts"                         
 [28] "HTert_ex9vex20_over"                      
 [29] "HTert_ex9vex20_res"                       
 [30] "HTert_ex9vex20_resSig_padj"               
 [31] "HTert_ex9vex20_under"                     
 [32] "HTert_filt"                               
 [33] "HTert_genotype"                           
 [34] "HTert_norm_sigset"                        
 [35] "HTert_normalized_counts_f"                
 [36] "HTert_normalized_counts_f_nonames"        
 [37] "HTert_rlog"                               
 [38] "HTert_sampleDistMatrix"                   
 [39] "HTert_samples"                            
 [40] "HTert_vst"                                
 [41] "HiC_overlap_venns"                        
 [42] "IRS2_exp"                                 
 [43] "Lineage_vec"                              
 [44] "MCF10ADists"                              
 [45] "MCF10A_H1047R_counts"                     
 [46] "MCF10A_WTvex20_over"                      
 [47] "MCF10A_WTvex20_res"                       
 [48] "MCF10A_WTvex20_resSig_padj"               
 [49] "MCF10A_WTvex20_under"                     
 [50] "MCF10A_WTvex9_over"                       
 [51] "MCF10A_WTvex9_res"                        
 [52] "MCF10A_WTvex9_resSig_padj"                
 [53] "MCF10A_WTvex9_under"                      
 [54] "MCF10A_coldata"                           
 [55] "MCF10A_counts"                            
 [56] "MCF10A_dds"                               
 [57] "MCF10A_dds_f"                             
 [58] "MCF10A_diff_genes"                        
 [59] "MCF10A_ex9vex20_over"                     
 [60] "MCF10A_ex9vex20_res"                      
 [61] "MCF10A_ex9vex20_resSig_padj"              
 [62] "MCF10A_ex9vex20_under"                    
 [63] "MCF10A_filt"                              
 [64] "MCF10A_genotype"                          
 [65] "MCF10A_norm_sigset"                       
 [66] "MCF10A_normalized_counts_f"               
 [67] "MCF10A_normalized_counts_f_nonames"       
 [68] "MCF10A_rlog"                              
 [69] "MCF10A_sampleDistMatrix"                  
 [70] "MCF10A_samples"                           
 [71] "MCF10A_vst"                               
 [72] "MCF7_corrected_hic_enriched"              
 [73] "MCF7_parental_hic_enriched"               
 [74] "MCF_7_counts"                             
 [75] "MCF_dual_E545K_counts"                    
 [76] "MCF_total_coldata"                        
 [77] "MCF_total_dds"                            
 [78] "MCF_total_dds_f"                          
 [79] "MCF_total_samples"                        
 [80] "MCF_total_treatment"                      
 [

This is my total raw counts file from my Total RNA-seq environment

In [ ]:
total_counts

Here we create an object that defines the genotypes and lineages for each of our samples

In [ ]:
Lineage_vec <- c(rep("MCF10A", 6),
                  rep("cancer", 6),
                  rep("MCF10A", 3),
                  rep("HTert", 9),
                  rep("cancer", 3))
Lineage_vec
Genotype_vec <- c(rep("ex9", 3), rep("WT", 6), rep("ex9", 3), rep("ex20", 3),
                  rep("WT", 3), rep("ex9", 3), rep("ex20", 6))
Genotype_vec
sample_info <- data.frame(Lineage_vec, Genotype_vec)
rownames(sample_info) <- rownames(total_counts_t)
sample_info$Combo <- as.factor(paste(sample_info$Lineage_vec, sample_info$Genotype_vec))
names(sample_info) <- c("lineage", "genotype", "combo")
sample_info

In [ ]:
mn <- apply(total_counts_t,1,mean)
hist(log2(mn), breaks = 100)

In [ ]:
hist(log10(mn), breaks = 100)

Create a DESeq model with genotype and lineage as our variables

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = total_counts,
                              colData = sample_info,
                              design = ~lineage + genotype)
str(dds)

In [ ]:
vsd <- vst(dds, blind = FALSE)
head(assay(vsd), 3)

Here we create a heatmap that displays the similarities of our samples

In [ ]:
sampleDists <- dist(t(assay(vsd)))
sampleDists
sampleDistMatrix <- as.matrix( sampleDists )
rownames( sampleDistMatrix ) <- vsd$lineage
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
p <- pheatmap(sampleDistMatrix,
         clustering_distance_rows = sampleDists,
         clustering_distance_cols = sampleDists,
         col = colors)
p

In [ ]:
library(repr)
pdf(file = "dist_heatmap.pdf",
   width = 8,
   height = 8)
options(repr.plot.width=8, repr.plot.height=8)
p <- pheatmap(sampleDistMatrix,
         clustering_distance_rows = sampleDists,
         clustering_distance_cols = sampleDists,
         col = colors)
p
dev.off

Our PCA's show differences between our cell lines not defined by genotype, but rather by lineage

In [ ]:
plotPCA(vsd, intgroup = c("genotype"))
plotPCA(vsd, intgroup = c("lineage"))
plotPCA(vsd, intgroup = c("combo"))

In [ ]:
pdf(file="genotype.pca.pdf")
plotPCA(vsd, intgroup = c("genotype"))
dev.off()

pdf(file="lineage.pca.pdf")
plotPCA(vsd, intgroup = c("lineage"))
dev.off()

pdf(file="combo.pca.pdf")
plotPCA(vsd, intgroup = c("combo"))
dev.off()

In [ ]:
pca <- prcomp(total_counts)

In [ ]:
library(pca3d)
gr <- sample_info$lineage
options(rgl.useNULL = TRUE)
pca3d(pca, group=gr)

In [ ]:
dds <- DESeq(dds)
res <- results(dds)
res <- res[order(res$padj),]
head(res)

plotMA(dds,ylim=c(-2,2),main="DESeq2")

## GSVA

In [ ]:
myC6 <- getGmt("/data/park_lab/adam/Total_RNA/c6.all.v7.5.1.entrez.gmt")

In [ ]:
library('limma')

In [ ]:
dat  <- counts(dds, normalized = TRUE)
idx  <- rowMeans(dat) > 1
dat  <- dat[idx, ]
mod  <- model.matrix(~genotype, colData(dds))
mod0 <- model.matrix(~1, colData(dds))
svseq <- svaseq(dat, mod, mod0)

In [ ]:
ddssva <- dds
ddssva$SV1 <- svseq$sv[,1]
ddssva$SV2 <- svseq$sv[,2]
ddssva$SV3 <- svseq$sv[,3]
design(ddssva) <- ~SV1 + SV2 + SV3 + genotype
ddssva <- DESeq(ddssva)

In [ ]:
vsd_sva <- vst(ddssva, blind = FALSE)
head(assay(vsd_sva), 3)

In [ ]:
plotPCA(vsd_sva, intgroup = c("genotype"))

In [ ]:
resCombo <- results(ddssva,
                    contrast=c("genotype", "ex9", "ex20"))

head(resCombo)
table(resCombo$padj < 0.05)

In [ ]:
cov = cbind(svseq$sv[,1:3])
counts_deseq_sva <- removeBatchEffect(dat, covariates = cov)

In [ ]:
counts_deseq_sva

In [ ]:
cleaningY = function(y, mod, svaobj) {
X=cbind(mod,svaobj$sv)
Hat=solve(t(X)%*%X)%*%t(X)
 beta=(Hat%*%t(y))
P=ncol(mod)
cleany=y-t(as.matrix(X[,-c(1:P)])%*%beta[-c(1:P),])
 return(cleany)
}

In [ ]:
cleany = cleaningY(dat,mod,svseq)

In [ ]:
cleany

In [ ]:
good_names <- rownames(cleany)
good_names <- tools::file_path_sans_ext(c(good_names))
good_names
rownames(cleany) <- good_names
head(cleany)

In [ ]:
genes2 <- mapIds(org.Hs.eg.db,
                     keys=rownames(cleany),
                     column="ENTREZID",
                     keytype="ENSEMBL",
                     multiVals="first")

In [ ]:
rownames(cleany) = genes2

In [ ]:
cleany

In [ ]:
nas=na.omit(rownames(cleany))

In [ ]:
cleany2sss=cleany[-na.action(nas),]

In [ ]:
dups <- which(duplicated(rownames(cleany2)))

In [ ]:
cleany3=cleany2[-dups,]

In [ ]:
cleany3

In [ ]:
colnames(cleany3)

In [ ]:
sub = c(1,2,3,10,11,12,13,14,15,19,20,21,22,23,24,25,26,27)
exampleSet <- ExpressionSet(assayData = cleany3[,sub])
test = gsva(exprs(exampleSet), myC6, method = "ssgsea",
            kcdf = "Poisson", verbose=TRUE,
            parallel.sz=1, ssgsea.norm = F)

In [ ]:
myvars <- apply(test,1, var,na.rm=TRUE) 
myvars <- sort(myvars,decreasing=TRUE) 
length(myvars)

In [ ]:
expression <- test[names(myvars),] 
dim(expression)

In [ ]:
clinsub=sample_info[sub,]
clinsub

In [ ]:
lp = as.factor(clinsub[,3])
secondp = as.factor(clinsub[,2])

In [ ]:
expression

In [ ]:
bp=brewer.pal(10,"Spectral")
lp = as.factor(clinsub[,3])
secondp = as.factor(clinsub[,2])
#grid.newpage()
pdf("~/E545KvH1047R.ssgsea.pdf")
my_palette <- colorRampPalette(c("purple", "white", "green"))(n = 1000)
heatmap.2(expression,
          trace = "none",
          col = my_palette,
          margins = c(5,25),
          labCol = "",
          cexRow = .7, ColSideColors = bp[lp]
          key = TRUE, scale = "row")
legend("topright",legend = unique(lp), fill = bp[unique(lp)])
dev.off()

In [ ]:
geneSetClustering <- hclust(as.dist(1-cor(t(expression), method="kendall")),
                            method="complete")
heatmap.2(expression,
          trace = "none",
          col = my_palette,
          margins = c(5,25),
          labCol = "",
          cexRow = .7, ColSideColors = bp[lp],
          key = TRUE, scale = "row",
          Rowv=as.dendrogram(geneSetClustering))
legend("topright",legend = unique(lp), fill = bp[unique(lp)])

In [ ]:
pdf("~/E545KvH1047R_clusteredy.ssgsea.pdf")
my_palette <- colorRampPalette(c("purple", "white", "green"))(n = 1000)
heatmap.2(expression,
          trace = "none",
          col = my_palette,
          margins = c(5,25),
          labCol = "",
          cexRow = .7, ColSideColors = bp[lp],
          key = TRUE, scale = "row",
         Rowv=as.dendrogram(geneSetClustering))
legend("topright",legend = unique(lp), fill = bp[unique(lp)])
dev.off()

In [ ]:
heatmap.2(expression,
          trace = "none",
          col = my_palette,
          margins = c(5,25),
          labCol = "",
          cexRow = .7, ColSideColors = bp[lp],
          key = TRUE, scale = "row",
         )
legend("topright",legend = unique(lp), fill = bp[unique(lp)])

In [ ]:
CRISPR_guides <- read_tsv(file = "MCF10A_guides.txt")
CRISPR_guides

In [ ]:
Gene_targets <- CRISPR_guides %>%
                dplyr::select(Target.Gene.Symbol) %>%
                unique() %>%
                pull(Target.Gene.Symbol) %>%
                sort()
Gene_targets

In [ ]:
test_df <- as.data.frame(test) %>%
            tibble::rownames_to_column("PATHWAY")
number_col <- seq(1, 189, by=1)
test_df$Number_ID <- number_col
test_df

In [ ]:
TGFB_UP <- test_df %>% 
            dplyr::filter(PATHWAY == "TGFB_UP.V1_UP")
TGFB_DN <- test_df %>% 
            dplyr::filter(PATHWAY == "TGFB_UP.V1_DN")
TGFB_UP
TGFB_DN

In [ ]:
TGFB_UP_genes <- myC6[[167]]@geneIds
TGFB_DN_genes <- myC6[[67]]@geneIds
str(TGFB_UP_genes)
str(TGFB_DN_genes)

In [ ]:
TGFB_DN_guides <- CRISPR_guides %>% 
        filter(Target.Gene.ID %in% TGFB_DN_genes)
TGFB_DN_guides
TGFB_DN_targets <- TGFB_DN_guides %>% 
                pull(Target.Gene.Symbol) %>%
                unique() %>%
                sort()
TGFB_DN_targets
TGFB_UP_guides <- CRISPR_guides %>% 
        filter(Target.Gene.ID %in% TGFB_UP_genes)
TGFB_UP_guides
TGFB_UP_targets <- TGFB_UP_guides %>% 
                pull(Target.Gene.Symbol) %>%
                unique() %>%
                sort()
TGFB_UP_targets

### Heatmap Cleanup

In [ ]:
expression

In [ ]:
colMeta_df <- data.frame(Genotype = c(rep("E545K", 6), rep("H1047R", 3), rep("E545K", 3), rep("H1047R", 6)), 
                      Lineage = c(rep("MCF10A", 3), rep("MCF7", 3), rep("MCF10A", 3),
                                 rep("HTert-IMEC", 6), rep("T47D", 3)),
                      stringsAsFactors = F, 
                      row.names = colnames(expression))
colMeta_df

In [ ]:
p <- pheatmap(expression,
              clustering_distance_cols = "euclidean", 
              clustering_distance_rows = "euclidean", 
              annotation_col = colMeta_df,
              fontsize_number = 2, 
              scale = "row", 
              cutree_cols = 2,
              show_colnames = FALSE
             )
p

In [ ]:
add.flag <- function(pheatmap,
                     kept.labels,
                     repel.degree) {

  # repel.degree = number within [0, 1], which controls how much 
  #                space to allocate for repelling labels.
  ## repel.degree = 0: spread out labels over existing range of kept labels
  ## repel.degree = 1: spread out labels over the full y-axis

  heatmap <- pheatmap$gtable

  new.label <- heatmap$grobs[[which(heatmap$layout$name == "row_names")]] 

  # keep only labels in kept.labels, replace the rest with ""
  new.label$label <- ifelse(new.label$label %in% kept.labels, 
                            new.label$label, "")

  # calculate evenly spaced out y-axis positions
  repelled.y <- function(d, d.select, k = repel.degree){
    # d = vector of distances for labels
    # d.select = vector of T/F for which labels are significant

    # recursive function to get current label positions
    # (note the unit is "npc" for all components of each distance)
    strip.npc <- function(dd){
      if(!"unit.arithmetic" %in% class(dd)) {
        return(as.numeric(dd))
      }

      d1 <- strip.npc(dd$arg1)
      d2 <- strip.npc(dd$arg2)
      fn <- dd$fname
      return(lazyeval::lazy_eval(paste(d1, fn, d2)))
    }

    full.range <- sapply(seq_along(d), function(i) strip.npc(d[i]))
    selected.range <- sapply(seq_along(d[d.select]), function(i) strip.npc(d[d.select][i]))

    return(unit(seq(from = max(selected.range) + k*(max(full.range) - max(selected.range)),
                    to = min(selected.range) - k*(min(selected.range) - min(full.range)), 
                    length.out = sum(d.select)), 
                "npc"))
  }
  new.y.positions <- repelled.y(new.label$y,
                                d.select = new.label$label != "")
  new.flag <- segmentsGrob(x0 = new.label$x,
                           x1 = new.label$x + unit(0.15, "npc"),
                           y0 = new.label$y[new.label$label != ""],
                           y1 = new.y.positions)

  # shift position for selected labels
  new.label$x <- new.label$x + unit(0.2, "npc")
  new.label$y[new.label$label != ""] <- new.y.positions

  # add flag to heatmap
  heatmap <- gtable::gtable_add_grob(x = heatmap,
                                   grobs = new.flag,
                                   t = 4, 
                                   l = 4
  )

  # replace label positions in heatmap
  heatmap$grobs[[which(heatmap$layout$name == "row_names")]] <- new.label

  # plot result
  grid.newpage()
  grid.draw(heatmap)

  # return a copy of the heatmap invisibly
  invisible(heatmap)
}

In [ ]:
sort(rownames(expression))

In [ ]:
sets_of_int <- c("AKT_UP_MTOR_DN.V1_DN", "AKT_UP_MTOR_DN.V1_UP",
                "AKT_UP.V1_DN", "AKT_UP.V1_UP", 
                "ATF2_S_UP.V1_UP", "ATF2_UP.V1_DN", "ATF2_UP.V1_UP", 
                "JNK_DN.V1_DN", "JNK_DN.V1_UP", 
                "PTEN_DN.V1_DN", "PTEN_DN.V1_UP", "PTEN_DN.V2_DN", "PTEN_DN.V2_UP", 
                "MTOR_UP.V1_DN", "MTOR_UP.V1_UP", 
                "MYC_UP.V1_DN", "MYC_UP.V1_UP", 
                "TGFB_UP.V1_DN", "TGFB_UP.V1_UP")

In [ ]:
p_improved <- add.flag(p, sets_of_int, 0)
p_improved

In [ ]:
ggsave(filename = "GSVA_heatmap_clean.pdf", plot = p_improved)

## Diff Seq Analyses

In [5]:
resultsNames(ddssva)

[1] "Intercept"            "SV1"                  "SV2"                 
[4] "SV3"                  "genotype_ex9_vs_ex20" "genotype_WT_vs_ex20"

### Genotype Direct Comparison

In [3]:
geno_comp <- results(ddssva, contrast = c("genotype", "ex9", "ex20"))
geno_comp <- as.data.frame(geno_comp)
geno_comp_filt <- geno_comp %>%
  filter(log2FoldChange < -1.5 | log2FoldChange > 1.5) %>%
  filter(padj < 0.05)
geno_comp_filt

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000278791.1,6.612653,-2.302249,0.4649189,-4.951937,7.347834e-07,7.714498e-06
ENSG00000229376.3,12.801501,-1.756743,0.4287367,-4.097487,4.176587e-05,2.741731e-04
ENSG00000142609.18,13.785815,1.869731,0.3283950,5.693544,1.244291e-08,1.983657e-07
ENSG00000187730.9,29.824951,2.674128,0.5126600,5.216183,1.826478e-07,2.238148e-06
ENSG00000238164.6,40.461046,-1.773661,0.4521381,-3.922831,8.751443e-05,5.237764e-04
ENSG00000157873.17,232.385304,-1.795708,0.3865943,-4.644942,3.401715e-06,2.984972e-05
ENSG00000177133.11,10.966659,4.003949,0.8200935,4.882308,1.048511e-06,1.060372e-05
ENSG00000142611.17,68.509740,4.330422,0.6370000,6.798151,1.059707e-11,3.358566e-10
ENSG00000162591.16,8307.720221,1.723491,0.3207743,5.372910,7.747601e-08,1.024647e-06


In [4]:
prePathfindR_df <- geno_comp_filt %>%
                    rownames_to_column("GeneID") %>%
                    dplyr::select(c("GeneID", "log2FoldChange", "padj"))

GeneID,log2FoldChange,padj
<chr>,<dbl>,<dbl>
ENSG00000278791.1,-2.302249,7.714498e-06
ENSG00000229376.3,-1.756743,2.741731e-04
ENSG00000142609.18,1.869731,1.983657e-07
ENSG00000187730.9,2.674128,2.238148e-06
ENSG00000238164.6,-1.773661,5.237764e-04
ENSG00000157873.17,-1.795708,2.984972e-05
ENSG00000177133.11,4.003949,1.060372e-05
ENSG00000142611.17,4.330422,3.358566e-10
ENSG00000162591.16,1.723491,1.024647e-06


In [3]:
prePathfindR_df
write_tsv(prePathfindR_df, file = "GenoDirectComp_SVA.txt")

hgnc_symbol,log2FoldChange,padj
<chr>,<dbl>,<dbl>
CFH,2.000816,2.186195e-07
AOC1,-2.576799,5.349419e-03
WNT16,3.088030,1.045158e-06
TFPI,1.910179,6.260387e-10
SLC7A2,3.866093,2.657544e-24
ABCB5,1.632194,3.167950e-02
SKAP2,2.119249,2.514852e-11
MPO,2.904284,1.020554e-03
SOX8,3.605927,2.937395e-05


In [5]:
library(biomaRt)

In [6]:
#add gene symbols
prePathfindR_df$GeneID <- prePathfindR_df$GeneID %>% 
  tools::file_path_sans_ext()
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
genes <- prePathfindR_df$GeneID
G_list <- getBM(filters= "ensembl_gene_id",
                attributes= c("ensembl_gene_id","hgnc_symbol"),
                values=genes,mart= mart,
               useCache = FALSE)
prePathfindR_df <- merge(prePathfindR_df,
                               G_list,
                               by.x="GeneID",
                               by.y="ensembl_gene_id")
prePathfindR_df

GeneID,log2FoldChange,padj,hgnc_symbol
<chr>,<dbl>,<dbl>,<chr>
ENSG00000000971,2.000816,2.186195e-07,CFH
ENSG00000002726,-2.576799,5.349419e-03,AOC1
ENSG00000002745,3.088030,1.045158e-06,WNT16
ENSG00000003436,1.910179,6.260387e-10,TFPI
ENSG00000003989,3.866093,2.657544e-24,SLC7A2
ENSG00000004846,1.632194,3.167950e-02,ABCB5
ENSG00000005020,2.119249,2.514852e-11,SKAP2
ENSG00000005381,2.904284,1.020554e-03,MPO
ENSG00000005513,3.605927,2.937395e-05,SOX8


In [7]:
prePathfindR_df <- prePathfindR_df %>%
                dplyr::select(c("hgnc_symbol", "log2FoldChange", "padj"))
prePathfindR_df

hgnc_symbol,log2FoldChange,padj
<chr>,<dbl>,<dbl>
CFH,2.000816,2.186195e-07
AOC1,-2.576799,5.349419e-03
WNT16,3.088030,1.045158e-06
TFPI,1.910179,6.260387e-10
SLC7A2,3.866093,2.657544e-24
ABCB5,1.632194,3.167950e-02
SKAP2,2.119249,2.514852e-11
MPO,2.904284,1.020554e-03
SOX8,3.605927,2.937395e-05


In [8]:
prePathfindR_df

hgnc_symbol,log2FoldChange,padj
<chr>,<dbl>,<dbl>
CFH,2.000816,2.186195e-07
AOC1,-2.576799,5.349419e-03
WNT16,3.088030,1.045158e-06
TFPI,1.910179,6.260387e-10
SLC7A2,3.866093,2.657544e-24
ABCB5,1.632194,3.167950e-02
SKAP2,2.119249,2.514852e-11
MPO,2.904284,1.020554e-03
SOX8,3.605927,2.937395e-05


In [9]:
paths_df <- run_pathfindR(prePathfindR_df, visualize_enriched_terms =FALSE)

`n_processes` is set to `iterations` because `iterations` < `n_processes`

There is already a directory named "pathfindR_Results".
Writing the result to "pathfindR_Results(7)" not to overwrite any previous results.

## Testing input

The input looks OK

## Processing input. Converting gene symbols,
          if necessary (and if human gene symbols provided)

Number of genes provided in input: 4367

Number of genes in input after p-value filtering: 4367

Warning message:
"Duplicated genes found! The lowest p value for each gene was selected"
pathfindR cannot handle p values < 1e-13. These were changed to 1e-13

Could not find any interactions for 1199 (37.66%) genes in the PIN

Final number of genes in input: 1980

## Performing Active Subnetwork Search and Enrichment

## Processing the enrichment results over all iterations

## Annotating involved genes and visualizing enriched terms

## Creating HTML report



ERROR: Error in with_pandoc_safe_environment(system(paste(shQuote(path), "--version"), : The 'HOME' environment variable must be set before running Pandoc.


In [10]:
paths_df

ERROR: Error in eval(expr, envir, enclos): object 'paths_df' not found


In [ ]:
p <- ggplot(paths_df, aes(x= Fold_Enrichment, y = support, color = support)) +
            geom_point() + 
            scale_size(paths_df$occurrence) +
            theme_classic() + 
            ggtitle("Speed Vs Dist coloured by speed")
p

In [ ]:
p

In [ ]:
save.image(file = "model_env.RData")